In [67]:
from icecream import ic
import numpy as np

from plasmapy.formulary import magnetostatics
from plasmapy.plasma.sources import Plasma3D

In [68]:
# PIC-CODE SMILEI
import numpy as np
from unyt import m, A, c, me, qp, mu_0
from unyt import unyt_array


filename = 'Fields_input.h5'

# Reference quantities
# ------------------

# --- arbitrary reference quantities
L_r = d_i = 1.3e5 * m  # reference length: ion inertial length (m)
w_r = c / L_r  # reference frequency
B_r = me * w_r / qp

# --- LMA field
M_d = 1.12e12
m_d = [0, M_d, 0] * A * m**2  # dipolar moment
s_d = -0.1 * d_i  # source depth


# --- geometry
dimensions = 3

# --- grid
L_x = 0.625 * d_i
L_y = 1.25 * d_i
L_z = 1.25 * d_i
grid_length = [L_x, L_y, L_z]
number_of_patches = [8, 16, 16]
cells_per_patch = [8] * dimensions
cell_length = [0.0] * dimensions
number_of_cells = [0.0] * dimensions
for i in range(dimensions):
    cell_length[i] = grid_length[i] / number_of_patches[i] / cells_per_patch[i]
    number_of_cells[i] = number_of_patches[i] * cells_per_patch[i]

# --- normalization
grid_length = [float(i / L_r) for i in grid_length]
cell_length = [float(i / L_r) for i in cell_length]

In [69]:
M_d = 1.12e12
m_d = [0, M_d, 0] * A * m**2

In [70]:
M_d = 1.12e12
m_d = [0, M_d, 0] * A * m**2
r_0 = unyt_array([s_d, L_y / 2, L_z / 2])

In [71]:
dipole = magnetostatics.MagneticDipole(
    m_d.to_astropy() , r_0.to_astropy()
)

def domain(grid_length, cell_length, number_of_cells, length_unit=1):
    return [np.linspace(0.-cell_length[i]*2, grid_length[i]+cell_length[i]*3, number_of_cells[i]+6)*length_unit for i in range(3)]

domain_x, domain_y, domain_z = domain(grid_length, cell_length, number_of_cells, L_r)

plasma = Plasma3D(
    domain_x=domain_x.to_astropy(), domain_y=domain_y.to_astropy(), domain_z=domain_z.to_astropy()
)
plasma.add_magnetostatic(dipole)

In [92]:
Bx = unyt_array.from_astropy(plasma.magnetic_field[0, :, :, :])/B_r  # because grid uses 'ij' indexing
By = plasma.magnetic_field[1, :, :, :].value/B_r.value  # because grid uses 'ij' indexing
Bz = plasma.magnetic_field[2, :, :, :].value/B_r.value  # because grid uses 'ij' indexing

In [ ]:
import h5py
filename = 'Fields_input.h5'
with h5py.File(filename, 'w') as hf:
    hf.create_dataset("Bx",  data=Bx)
    hf.create_dataset("By",  data=By)
    hf.create_dataset("Bz",  data=Bz)